In [1]:
%load_ext autoreload
%autoreload 2


Process pbp data


In [2]:
from nba import NbaTracker
from build_shot_chance_data import process_season
nbaTracker = NbaTracker()
# nbaTracker.add_season("2000-01")

In [15]:
from nba_api.stats.static.players import find_player_by_id
from nba_api.stats.static.teams import find_team_name_by_id


def get_player_names(event):
    for team, players in event.current_players.items():
        print(find_team_name_by_id(team)['nickname'], list(
            map(lambda pid: find_player_by_id(pid)['full_name'], players)))
    if hasattr(event, "player1_id"):
        print("player1", find_player_by_id(event.player1_id)['full_name'] if event.player1_id else 0)
    if hasattr(event, "player2_id"):
        print("player2", find_player_by_id(event.player2_id)['full_name'])
    if hasattr(event, "player3_id"):
        print("player3", find_player_by_id(event.player3_id)['full_name'])



In [3]:
season_name = "2015-16"
nbaTracker.add_season(season_name)
nbaTracker.load_possession_data(season_name)
games = nbaTracker.seasons[season_name].games


In [13]:
nbaTracker.seasons[season_name].build_player_seasons()

In [4]:

nbaTracker.add_season("2015-16")
season = nbaTracker.seasons["2015-16"]

try:
    games = process_season(season, 0)
    print("complete")
except Exception as e:
    eargs = e.args
    raise e

    # print("has exception")


prcessed 100 games
prcessed 200 games
prcessed 300 games
not adding delay of game fts
using missing delay of game tech
prcessed 400 games
prcessed 500 games
No expected free throws and no tech <StatsFreeThrow GameId: 0021500504, Description: Barton Free Throw Technical (9 PTS), Time: 12:00, EventNum: 283> adding another tech foul
No expected free throws and no tech <StatsFreeThrow GameId: 0021500571, Description: Thomas Free Throw Technical (16 PTS), Time: 12:00, EventNum: 256> adding another tech foul
prcessed 600 games
prcessed 700 games
prcessed 800 games
prcessed 900 games
prcessed 1000 games
prcessed 1100 games
prcessed 1200 games
complete


In [13]:
from build_shot_chance_data import EventType
from collections import defaultdict

total_count = 0
try_result_counts = defaultdict(int)
ft_count = 0
other_team_ft_count = 0
points = 0

for game in games:
    for ge in game.game_events:
        if ge.event_type == EventType.PossessionTry:
            try_result_counts[ge.result.result_class] += 1
            ft_count += ge.result.off_fts
            other_team_ft_count += ge.result.def_fts
            points += ge.result.points
            total_count += 1



In [14]:
for key, count in try_result_counts.items():
    print(key, count/total_count)
print(points / total_count)

ResultClass.REBOUND 0.3952953661124867
ResultClass.DEAD_TURNOVER 0.05322971070281668
ResultClass.MADE_SHOT 0.30815594720378425
ResultClass.FT 0.1019580099461079
ResultClass.LIVE_TURNOVER 0.06641489280537118
ResultClass.SAME_TEAM 0.07318434979226922
ResultClass.JUMPBALL 0.0017339797609882298
ResultClass.OTHER_TEAM_FT 2.7743676175811677e-05
0.7250913807334041


In [7]:
points

0

# Predict free throw chance
player ft%, last 3 years ft%, weighted sum?, shooting %?

# live ft chance
player ft%, score margin, time left, other players?

In [11]:
from build_shot_chance_data import EventType
from collections import defaultdict

ft_result_counts = defaultdict(int)

ft_count = 0
made_fts = 0
live_fts = 0
for game in games:
    for ft in game.free_throws:
        ft_count += 1
        if ft.is_made:
            made_fts+= 1
    for ge in game.game_events:
        if ge.event_type == EventType.LiveFreeThrow:
            ft_count += 1
            live_fts += 1
            ft_result_counts[ge.result.result_class] += 1
            if ge.result.is_made:
                made_fts += 1

            


In [12]:
for key, count in ft_result_counts.items():
    print(key, count/live_fts)
print(made_fts / ft_count)


ResultClass.REBOUND 0.2296758767268863
ResultClass.MADE_SHOT 0.7662393730074389
ResultClass.DEAD_TURNOVER 0.0026899574920297557
ResultClass.FT 0.0010626992561105207
ResultClass.JUMPBALL 0.00016604675876726887
ResultClass.OTHER_TEAM_FT 0.00013283740701381508
ResultClass.SAME_TEAM 3.320935175345377e-05
0.7559360333738919


# Make sure fts right

In [5]:
from collections import Counter
from build_shot_chance_data import EventType, LiveFreeThrowResult
g = games[-1]
made = 0
tech_num = 0
c = Counter()
for ft in g.free_throws:
    if ft.is_made:
        made += 1
        # if ft.shooter == 200755:
        #     print(ft)
        c[ft.shooter] += 1
    if ft.is_tech:
        tech_num += 1
    # print(ft.is_tech)
print(made, len(g.free_throws), tech_num)

game_events = g.game_events
made = 0
total = 0


for i in range(len(game_events)):
    e = game_events[i]
    if e.event_type == EventType.LiveFreeThrow:
        total += 1
        if e.result.ft_result == LiveFreeThrowResult.MADE:
            # if e.result.shooter == 200755:
            #     print(e)
            c[e.result.shooter] += 1
            made += 1
        print(e.result.ft_result)
    elif (e.event_type == EventType.PossessionTry or e.event_type == EventType.Rebound) and e.result.num_fts != 0:
        print( e.result.num_fts, e.period, e.period_time_left //
              60, e.period_time_left % 60)
    # elif e.event_type == EventType.Rebound:
    #     print("reb", e.result.num_fts)
print(made, total)


14 19 1
3 1 4 2
LiveFreeThrowResult.MADE
2 1 3 18
LiveFreeThrowResult.MADE
2 1 2 0
LiveFreeThrowResult.MISS
1 1 1 24
LiveFreeThrowResult.MADE
2 1 1 1
LiveFreeThrowResult.MADE
2 1 0 50
LiveFreeThrowResult.MADE
2 1 0 6
LiveFreeThrowResult.MADE
1 2 10 33
2 2 9 5
LiveFreeThrowResult.MADE
3 2 7 21
LiveFreeThrowResult.MADE
2 2 4 35
LiveFreeThrowResult.MADE
2 2 1 39
LiveFreeThrowResult.MISS
2 2 1 14
LiveFreeThrowResult.MADE
1 3 4 47
LiveFreeThrowResult.MADE
2 3 4 29
LiveFreeThrowResult.MADE
1 3 3 38
LiveFreeThrowResult.MISS
1 4 11 9
LiveFreeThrowResult.MADE
2 4 10 15
LiveFreeThrowResult.MADE
2 4 8 3
LiveFreeThrowResult.MADE
1 4 5 41
LiveFreeThrowResult.MISS
2 4 4 0
LiveFreeThrowResult.MADE
2 4 0 25
LiveFreeThrowResult.MISS
16 21


In [17]:
for k in c:
    print(find_player_by_id(k)['full_name'], c[k])

Stephen Curry 7
Eric Gordon 2
Anthony Davis 10
Marreese Speights 4
Draymond Green 2
Klay Thompson 3
Ryan Anderson 4
Ish Smith 2
James Michael McAdoo 2
Harrison Barnes 1
Festus Ezeli 1
Dante Cunningham 1


In [23]:
c

Counter({203463: 1,
         201933: 11,
         201158: 6,
         202326: 1,
         200755: 4,
         201599: 5,
         101108: 6,
         200765: 2,
         2037: 4,
         201954: 2,
         200752: 2,
         1626161: 2,
         202362: 1,
         201956: 1})

22

(24, 36)

In [7]:
g.technicals

({}, {}, 0)

In [24]:

nbaTracker.add_season("2016-17")
season = nbaTracker.seasons["2016-17"]

try:
    games2 = process_season(season, 0)
    print("complete")
except Exception as e:
    eargs = e.args
    # raise e


prcessed 100 games
corrected jumpball to 1610612740
prcessed 200 games
prcessed 300 games
prcessed 400 games
corrected jumpball to 1610612738
prcessed 500 games
not adding delay of game fts
prcessed 600 games
prcessed 700 games
prcessed 800 games
No expected free throws and no tech <StatsFreeThrow GameId: 0021600854, Description: MISS Curry Free Throw Technical, Time: 12:00, EventNum: 305> adding another tech foul
prcessed 900 games
corrected jumpball to 1610612746
prcessed 1000 games
prcessed 1100 games
No expected free throws and no tech <StatsFreeThrow GameId: 0021601119, Description: Redick Free Throw Technical (28 PTS), Time: 4:57, EventNum: 356> adding another tech foul
prcessed 1200 games
complete


In [53]:
nbaTracker.add_season("2017-18")
season = nbaTracker.seasons["2017-18"]

try:
    games3 = process_season(season, 197)
    print("complete")
except Exception as e:
    eargs = e.args
    # raise e
    


Already loaded 2017-18
w fts 0 <StatsFoul GameId: 0021700200, Description: Covington Personal Take Foul (P3.PN) (Z.Zarba), Time: 1:42, EventNum: 491>
just ignore missed foul
corrected jumpball to 1610612746
prcessed 100 games
No expected free throws and no tech <StatsFreeThrow GameId: 0021700319, Description: Miles Free Throw Technical (4 PTS), Time: 12:00, EventNum: 311> adding another tech foul
prcessed 200 games
corrected jumpball to 1610612754
prcessed 300 games
wrong offense team for ft <StatsFreeThrow GameId: 0021700507, Description: Powell Free Throw 1 of 2 (3 PTS), Time: 2:17, EventNum: 100>
corrected jumpball to 1610612744
not adding delay of game fts
prcessed 400 games
prcessed 500 games
corrected jumpball to 1610612766
corrected jumpball to 1610612749
prcessed 600 games
corrected jumpball to 1610612748
wrong team lftv <StatsViolation GameId: 0021700882, Description: Napier Violation:Lane (.), Time: 3:13, EventNum: 486>
just ignoring...
prcessed 700 games
corrected jumpball t

In [56]:
nbaTracker.add_season("2018-19")
season = nbaTracker.seasons["2018-19"]

try:
    games4 = process_season(season, 155)
    print("complete")
except Exception as e:
    eargs = e.args
    # raise e


Already loaded 2018-19
prcessed 100 games
prcessed 200 games
corrected jumpball to 1610612746
prcessed 300 games
No expected free throws and no tech <StatsFreeThrow GameId: 0021800534, Description: Lillard Free Throw Technical (16 PTS), Time: 12:00, EventNum: 340> adding another tech foul
lane violation w exp fts 2 <StatsViolation GameId: 0021800546, Description: Covington Lane Violation, Time: 3:46, EventNum: 312>
[<StatsFoul GameId: 0021800546, Description: S. Hill P.FOUL (P2.PN) (K.Fitzgerald), Time: 3:46, EventNum: 261>, <StatsViolation GameId: 0021800546, Description: Covington Lane Violation, Time: 3:46, EventNum: 312>, <StatsFreeThrow GameId: 0021800546, Description: Towns Free Throw 2 of 2 (6 PTS), Time: 3:46, EventNum: 313>]
prcessed 400 games
corrected jumpball to 1610612743
corrected jumpball to 1610612737
not adding delay of game fts
prcessed 500 games
No expected free throws and no tech <StatsFreeThrow GameId: 0021800676, Description: MISS Durant Free Throw Technical, Time

In [57]:
nbaTracker.add_season("2019-20")
season = nbaTracker.seasons["2019-20"]

try:
    games5 = process_season(season, 0 )
    print("complete")
except Exception as e:
    eargs = e.args
    # raise e


Already loaded 2019-20
prcessed 100 games
prcessed 200 games
prcessed 300 games
corrected jumpball to 1610612743
No expected free throws and no tech <StatsFreeThrow GameId: 0021900325, Description: Poole Free Throw Technical (1 PTS), Time: 0:28, EventNum: 445> adding another tech foul
prcessed 400 games
made shot with  2 fts foul? <StatsFoul GameId: 0021900487, Description: Valanciunas FLAGRANT.FOUL.TYPE1 (P2.PN) (K.Fitzgerald), Time: 0:10, EventNum: 346>
prcessed 500 games
prcessed 600 games
prcessed 700 games
No expected free throws and no tech <StatsFreeThrow GameId: 0021900742, Description: Morris Sr. Free Throw Technical (5 PTS), Time: 12:00, EventNum: 311> adding another tech foul
prcessed 800 games
prcessed 900 games
corrected jumpball to 1610612738
prcessed 1000 games
corrected jumpball to 1610612755
lane violation w exp fts 2 <StatsViolation GameId: 0021901309, Description: Kurucs Lane Violation, Time: 4:53, EventNum: 671>
[<StatsFoul GameId: 0021901309, Description: Whiteside

In [69]:
nbaTracker.add_season("2020-21")
season = nbaTracker.seasons["2020-21"]

try:
    games6 = process_season(season, 444)
    print("complete")
except Exception as e:
    eargs = e.args
    # raise e


Already loaded 2020-21
not adding delay of game fts
prcessed 100 games
prcessed 200 games
prcessed 300 games
made shot with  2 fts foul? <StatsFoul GameId: 0022000768, Description: Booker FLAGRANT.FOUL.TYPE1 (P2.PN) (M.Kogut), Time: 1:49, EventNum: 285>
prcessed 400 games
corrected jumpball to 1610612756
prcessed 500 games
lane violation with just techs <StatsViolation GameId: 0022000970, Description: Lane, Time: 10:23, EventNum: 189> not sure what this means, ignoring
unhandled loose ball foul turnover [<StatsFoul GameId: 0022000971, Description: Olynyk L.B.FOUL (P2.T2) (C.Kirkland), Time: 6:57, EventNum: 377>, <StatsTimeout GameId: 0022000971, Description: ROCKETS Timeout: Regular (Full 3 Short 0), Time: 6:57, EventNum: 380>, <StatsSubstitution GameId: 0022000971, Description: SUB: Gibson FOR Noel, Time: 6:57, EventNum: 384>, <StatsJumpBall GameId: 0022000971, Description: Jump Ball Wood vs. Randle: Tip to Payton, Time: 6:52, EventNum: 386>]
prcessed 600 games
No expected free throws

In [66]:
eargs[1][-1].try_result

TryResult(result_type=<TryResultType.MISTAKE_CALL: 57>, result_player1_id=None, result_player2_id=None, result_player3_id=None, result_player4_id=None, shot_type=None, shot_distance=None, shot_X=None, shot_Y=None, num_fts=0)

In [47]:
find_team_name_by_id(1610612748)


{'id': 1610612748,
 'full_name': 'Miami Heat',
 'abbreviation': 'MIA',
 'nickname': 'Heat',
 'city': 'Miami',
 'state': 'Florida',
 'year_founded': 1988}

In [59]:
eargs[0].previous_possession.previous_possession.events


[<StatsFieldGoal GameId: 0022001063, Description: MISS Edwards 26' 3PT Jump Shot, Time: 0:53, EventNum: 679>,
 <StatsRebound GameId: 0022001063, Description: Nowell REBOUND (Off:1 Def:1), Time: 0:48, EventNum: 680>,
 <StatsFieldGoal GameId: 0022001063, Description: Nowell 1' Putback Layup (7 PTS), Time: 0:48, EventNum: 681>]

In [67]:
eargs[0].previous_possession.events


[<StatsFoul GameId: 0022000445, Description: Forbes S.FOUL (P1.PN) (S.Wall), Time: 2:41, EventNum: 629>,
 <StatsFreeThrow GameId: 0022000445, Description: Siakam Free Throw 1 of 2 (26 PTS), Time: 2:41, EventNum: 631>,
 <StatsFreeThrow GameId: 0022000445, Description: Siakam Free Throw 2 of 2 (27 PTS), Time: 2:41, EventNum: 632>]

In [68]:
eargs[0].events

[<StatsFoul GameId: 0022000445, Description: VanVleet P.FOUL (P4.PN) (S.Foster), Time: 2:35, EventNum: 691>,
 <StatsTimeout GameId: 0022000445, Description: Raptors Timeout: Coach Challenge (Reg.4 Short 0), Time: 2:35, EventNum: 678>,
 <StatsReplay GameId: 0022000445, Description: Instant Replay4th Period (10:11 PM EST), Time: 2:35, EventNum: 636>,
 <StatsJumpBall GameId: 0022000445, Description: Jump Ball (CC) Antetokounmpo vs. Anunoby: Tip to Forbes, Time: 2:33, EventNum: 637>,
 <StatsFieldGoal GameId: 0022000445, Description: MISS Antetokounmpo 2' Layup: Siakam BLOCK (2 BLK), Time: 2:21, EventNum: 640>,
 <StatsRebound GameId: 0022000445, Description: Anunoby REBOUND (Off:0 Def:7), Time: 2:17, EventNum: 642>]

In [18]:
eargs[0].next_possession.events

[<StatsViolation GameId: 0021700692, Description: Cousins Violation:Lane (K.Fitzgerald), Time: 10:30, EventNum: 167>,
 <StatsViolation GameId: 0021700692, Description: Portis Violation:Lane (.), Time: 10:30, EventNum: 215>,
 <StatsJumpBall GameId: 0021700692, Description: Jump Ball Cousins vs. Portis: Tip to Arcidiacono, Time: 10:30, EventNum: 231>,
 <StatsFieldGoal GameId: 0021700692, Description: MISS Portis 26' 3PT Jump Shot, Time: 10:11, EventNum: 168>,
 <StatsRebound GameId: 0021700692, Description: Nelson REBOUND (Off:0 Def:1), Time: 10:09, EventNum: 169>]

In [19]:
eargs[0].next_possession.next_possession.events


[<StatsFoul GameId: 0021700692, Description: Arcidiacono P.FOUL (P1.T2) (T.Ford), Time: 10:06, EventNum: 170>,
 <StatsFieldGoal GameId: 0021700692, Description: Cousins 11' Turnaround Jump Shot (8 PTS), Time: 9:51, EventNum: 172>]

In [21]:
from build_shot_chance_data import get_num_fta_from_foul
get_num_fta_from_foul(eargs[0].events[0])


2

In [5]:
from nba_dataclasses import TryResultType
counts = {i: 0 for i in TryResultType}
ft_nums = {i: {} for i in TryResultType}
next_result_same_team = {i: {True: 0, False: 0} for i in TryResultType}
result_player_teams = {i: {pi: {"off": 0, "def": 0} for pi in [1,2,3,4]} for i in TryResultType}


In [29]:
from nba_dataclasses import EventType, TryResultType, LiveFreeThrowResult
should_break = False
g_count = 0
for g in games: #+games2+games3+games4:
    g_count += 1
    game_events = g.game_events
    num_fts = 0
    ft_violations = 0
    expected_fts = 0
    for i in range(len(game_events)):
        e = game_events[i]
        if e.event_type == EventType.PossessionTry:
            expected_fts += e.result.num_fts
        elif e.event_type == EventType.Rebound:
            expected_fts += e.result.num_fts
        elif e.event_type == EventType.LiveFreeThrow:
            if e.result.ft_result in {LiveFreeThrowResult.DEF_LANE_VIOLATION_RETRY,}:
                ft_violations += 1
                
            else:
                num_fts += 1
    num_fts += len(g.free_throws)
    tech_num = len(g.technicals[0]) + \
        len(g.technicals[1]) + g.technicals[2]
    
    if num_fts != expected_fts+tech_num:
        print("not equal", num_fts, ft_violations, expected_fts, tech_num)
        break
                        # ptd = result_player_teams[tr.result_type]
        

        # if hasattr(tr, "result_player1_id") and tr.result_player1_id:
        #     if tr.result_player1_id in e.lineup.offense_players:
        #         ptd[1]['off'] += 1
        #     elif tr.result_player1_id in e.lineup.defense_players:
        #         ptd[1]['def'] += 1
        #     else:
        #         print("pid1 not in lineup")
        #         should_break = True
        #         break
            
        # if hasattr(tr, "result_player2_id") and tr.result_player2_id:
        #     if tr.result_player2_id in e.lineup.offense_players:
        #         ptd[2]['off'] += 1
        #     elif tr.result_player2_id in e.lineup.defense_players:
        #         ptd[2]['def'] += 1
        #     else:
        #         print("pid2 not in lineup")
        #         should_break = True
        #         break

        # if hasattr(tr, "result_player3_id") and tr.result_player3_id:
        #     if tr.result_player3_id in e.lineup.offense_players:
        #         ptd[3]['off'] += 1
        #     elif tr.result_player3_id in e.lineup.defense_players:
        #         ptd[3]['def'] += 1
        #     else:
        #         print("pid3 not in lineup")
        #         should_break = True
        #         break

        # if hasattr(tr, "result_player4_id") and tr.result_player4_id:
        #     if tr.result_player4_id in e.lineup.offense_players:
        #         ptd[4]['off'] += 1
        #     elif tr.result_player4_id in e.lineup.defense_players:
        #         ptd[4]['def'] += 1
        #     else:
        #         print("pid4 not in lineup")
        #         should_break = True
        #         break

        # next_t = tries[i]
        # is_same_team = t.lineup.offense_team == next_t.lineup.offense_team
        # next_result_same_team[t.try_result.result_type][is_same_team] += 1
        # counts[e.try_result.result_type] += 1
        # result_type_dic = ft_nums[e.try_result.result_type]
        # if e.try_result.num_fts in result_type_dic:
        #     result_type_dic[e.try_result.num_fts] += 1
        # else:
        #     result_type_dic[e.try_result.num_fts] = 1
    # if should_break:
    #     break


not equal 70 0 70 1


In [30]:
g_count

38

In [31]:
for i in range(len(game_events)):
    e = game_events[i]
    if e.event_type == EventType.LiveFreeThrow:
        print(e.result.ft_result)

LiveFreeThrowResult.MADE
LiveFreeThrowResult.MISS
LiveFreeThrowResult.MISS
LiveFreeThrowResult.MISS
LiveFreeThrowResult.MISS
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MISS
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MISS
LiveFreeThrowResult.MISS
LiveFreeThrowResult.MISS
LiveFreeThrowResult.MISS
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MISS
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MISS
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MISS
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE
LiveFreeThrowResult.MADE


In [13]:
g_count

28

In [20]:
len(g.free_throws)


14

In [19]:
for e in g.game_events:
    if e.event_type == EventType.LiveFreeThrow:
        if not e.result.shooter:
            print("no shooter")

In [8]:
result_player_teams


{<TryResultType.OTHER: 0>: {1: {'off': 0, 'def': 0},
  2: {'off': 0, 'def': 0},
  3: {'off': 0, 'def': 0},
  4: {'off': 0, 'def': 0}},
 <TryResultType.MISSED_SHOT: 1>: {1: {'off': 101792, 'def': 0},
  2: {'off': 0, 'def': 0},
  3: {'off': 0, 'def': 0},
  4: {'off': 0, 'def': 0}},
 <TryResultType.BLOCKED_SHOT: 2>: {1: {'off': 12193, 'def': 0},
  2: {'off': 0, 'def': 0},
  3: {'off': 0, 'def': 12193},
  4: {'off': 0, 'def': 0}},
 <TryResultType.MADE_SHOT: 3>: {1: {'off': 88358, 'def': 0},
  2: {'off': 52056, 'def': 0},
  3: {'off': 0, 'def': 0},
  4: {'off': 0, 'def': 0}},
 <TryResultType.AND1: 4>: {1: {'off': 5121, 'def': 0},
  2: {'off': 2509, 'def': 0},
  3: {'off': 0, 'def': 5121},
  4: {'off': 0, 'def': 0}},
 <TryResultType.DEF_GOALTEND_SHOT: 5>: {1: {'off': 500, 'def': 0},
  2: {'off': 228, 'def': 0},
  3: {'off': 130, 'def': 370},
  4: {'off': 0, 'def': 0}},
 <TryResultType.GOALTENDED_AND1: 6>: {1: {'off': 25, 'def': 0},
  2: {'off': 9, 'def': 0},
  3: {'off': 0, 'def': 25},
  4: 

In [74]:
ft_nums

{<TryResultType.OTHER: 0>: {},
 <TryResultType.MISSED_SHOT: 1>: {0: 383052},
 <TryResultType.BLOCKED_SHOT: 2>: {0: 44351},
 <TryResultType.MADE_SHOT: 3>: {0: 338120},
 <TryResultType.AND1: 4>: {1: 20320},
 <TryResultType.DEF_GOALTEND_SHOT: 5>: {0: 1839},
 <TryResultType.GOALTENDED_AND1: 6>: {1: 118},
 <TryResultType.MADE_BASKET_W_FOULSHOT: 7>: {1: 187},
 <TryResultType.MADE_BASKET_W_LOOSE_BALL_FOUL: 8>: {},
 <TryResultType.MADE_BASKET_W_FLAGRANT1: 67>: {1: 2},
 <TryResultType.MADE_BASKET_W_FLAGRANT2: 68>: {1: 2},
 <TryResultType.FLAGRANT1_AND1: 9>: {1: 15},
 <TryResultType.FLAGRANT2_AND1: 10>: {},
 <TryResultType.FLAGRANT1_AND1_2FTS: 63>: {2: 5},
 <TryResultType.NON_SHOOTING_FOUL: 11>: {0: 47679},
 <TryResultType.PENALTY_NON_SHOOTING_FOUL: 12>: {2: 14525},
 <TryResultType.SHOOTING_FOUL_2: 13>: {2: 65123},
 <TryResultType.SHOOTING_FOUL_3: 14>: {3: 3327},
 <TryResultType.AWAY_FROM_PLAY_FOUL: 15>: {1: 133},
 <TryResultType.AWAY_FROM_PLAY_0FTS_FOUL: 64>: {},
 <TryResultType.OFF_AWAY_FROM_P

In [15]:
counts

{<TryResultType.OTHER: 0>: 0,
 <TryResultType.MISSED_SHOT: 1>: 306603,
 <TryResultType.BLOCKED_SHOT: 2>: 35707,
 <TryResultType.MADE_SHOT: 3>: 269716,
 <TryResultType.AND1: 4>: 16126,
 <TryResultType.DEF_GOALTEND_SHOT: 5>: 1459,
 <TryResultType.GOALTENDED_AND1: 6>: 92,
 <TryResultType.MADE_BASKET_W_FOULSHOT: 7>: 151,
 <TryResultType.MADE_BASKET_W_LOOSE_BALL_FOUL: 8>: 0,
 <TryResultType.FLAGRANT1_AND1: 9>: 17,
 <TryResultType.FLAGRANT2_AND1: 10>: 0,
 <TryResultType.NON_SHOOTING_FOUL: 11>: 38792,
 <TryResultType.PENALTY_NON_SHOOTING_FOUL: 12>: 11588,
 <TryResultType.SHOOTING_FOUL_2: 13>: 52248,
 <TryResultType.SHOOTING_FOUL_3: 14>: 2601,
 <TryResultType.AWAY_FROM_PLAY_FOUL: 15>: 85,
 <TryResultType.OFF_AWAY_FROM_PLAY_FOUL: 16>: 4,
 <TryResultType.DEF_LOOSE_BALL_FOUL: 17>: 4,
 <TryResultType.PENALTY_DEF_LOOSE_BALL_FOUL: 18>: 262,
 <TryResultType.OFF_LOOSE_BALL_FOUL: 19>: 0,
 <TryResultType.PENALTY_OFF_LOOSE_BALL_FOUL: 20>: 21,
 <TryResultType.INBOUND_FOUL: 21>: 189,
 <TryResultType.CLEAR_

In [75]:
correct = {
    ("0021500248", 23): 1,
    ("0021500538", 354): 1,
    ("0021700035", 468): 1,
    ("0021700051", 68): 1,
    ("0021800546", 261): 2,
    ("0021700034", 237): 0,
    ("0021500542", 425): 0,
    ("0021601152", 528): 3,
    ("0021500171", 241): 2,
    ("0021500171", 242): 1,
    ("0021700144", 294): 2,
    ("0021700264", 260): 1,
    ("0021700684", 520): 1,
    ("0021700692", 164): 2,
    ("0021800546", 261): 2,
    ("0021900035", 242): 2,
    ("0021900635", 558): 1,
    ("0022000172", 329): 1,
    ("0022000268", 628): 2,
    ("0022001063", 682): 2,
    ("0021700536", 462): 2,
    ("0021700582", 159): 2,
    ("0021700582", 165): 3,
    ("0021700692", 164): 2,
    
    ("0021700020", 665): 0,
    ("0021700034", 237): 0,
    ("0021700050", 159): 2,  # pbp looks like 1? but should be 2?...
    ("0021700054", 54): 2,
    ("0021700090", 408): 2,
    ("0021700095", 397): 1,
    ("0021700099", 285): 2,
    ("0021700103", 584): 1,
    ("0021700112", 229): 0,
    ("0021700132", 465): 2,
    ("0021700136", 456): 2,
    ("0021700136", 459): 2,
    ("0021700136", 529): 2,
    ("0021700146", 78): 1,
    ("0021700157", 554): 1,
    ("0021700167", 70): 2,
    ("0021700183", 271): 0,
    ("0021700198", 234): 3,
    ("0021700220", 690): 0,
    ("0021700229", 610): 2,
    ("0021700231", 19): 1,
    ("0021700252", 625): 1,
    ("0021700257", 626): 0, # duplicate event, second one gets the fts
    ("0021700284", 327): 1,
    ("0021700312", 726): 2,
    ("0021700330", 140): 2,
    
    ("0021700692", 164): 2,
    
    
    ("0021800004", 671): 0,
    ("0021800010", 39): 1,
    ("0021800022", 249): 1,
    ("0021800028", 158): 2, 
    ("0021800033", 348): 1,
    ("0021800033", 351): 1,
    
}

In [18]:
from pbpstats.client import Client
from build_shot_chance_data import get_num_fta_from_foul, get_num_flagrant_fts
from pbpstats.resources import enhanced_pbp
import os
game_possessions_settings = {
    "dir": "C:/Users/bhalb/nbaproject/response_data",
    "Boxscore": {"source": "file", "data_provider": "stats_nba"},
    "Possessions": {"source": "file", "data_provider": "stats_nba"},
}

possession_client = Client(game_possessions_settings)

file_directory = "C:/Users/bhalb/nbaproject/response_data/pbp"
count = 0
target_count = 5
pass_count = 0
fail_count = 0 
process_count = 0

nbaTracker.add_season("2014-15")
season = nbaTracker.seasons["2014-15"]

for game_dict in season.schedule.games.final_games:
    game_id = game_dict['game_id']
    process_count += 1
    # print("processing game", game_id)
    # game_id = "0021600035"
    
    # data_pbp_file_path = (
    #     f"{file_directory}/data_{game_id}.json"
    # )
    # if not os.path.isfile(data_pbp_file_path):
    #     print("load", data_pbp_file_path)
    #     game = possession_client.Game(game_id)
    #     print("loaded!")
    # else:
    #     print(data_pbp_file_path, "already loaded")
    # pass_count += 1
    # break
    # if pass_count == 5:
    #     break
    game = possession_client.Game(game_id)
    possessions = game.possessions.items
    
    for possession in possessions:

        # print(count)
        # if count == 56:
        #     print(event)
        for event in possession.events:
            # if isinstance(event, enhanced_pbp.Turnover):
            #     if "Lane Violation" in event.description:
            #         print(event.is_lane_violation, event.event_action_type)
            if isinstance(event, enhanced_pbp.Foul) and not (event.is_technical or event.is_double_technical):
                # if "IN.FOUL" in event.description or "Inbound Foul" in event.description:
                #     continue
                # nft = event.number_of_fta_for_foul
                # nft = nft if nft is not None else 0
                ftnum = get_num_fta_from_foul(event)
                if event.is_flagrant:
                    num_flagrant_fts = get_num_flagrant_fts(event)
                    if num_flagrant_fts == 0:
                        ftnum = min(ftnum, 2)
                    else:
                        ftnum = num_flagrant_fts
                if (game_id, event.event_num) in correct:
                    if correct[(game_id, event.event_num)] == ftnum:
                        # print("pass")
                        pass_count += 1
                    else:
                        fail_count += 1
                        print("fail", ftnum, "vs", correct[(
                            game_id, event.event_num)], event, possession.period)
                        count = target_count
                        break
                # elif ftnum != nft:
                #     count += 1
                #     if count > target_count - 5:
                #         print("fouls: ", ftnum, "vs", nft, event, possession.period)
                #     break 
        if count == target_count:
            break
    if count == target_count:
        break
    if process_count % 100 == 0:
        print("processed", process_count, "passed:", pass_count)
    

# print("fail count:", fail_count)


processed 100 passed: 0
processed 200 passed: 2
processed 300 passed: 3
processed 400 passed: 3
processed 500 passed: 3
processed 600 passed: 5
processed 700 passed: 5
processed 800 passed: 5
processed 900 passed: 5
processed 1000 passed: 5
processed 1100 passed: 5
processed 1200 passed: 5


In [13]:
ne = event.next_event
ne

<StatsFoul GameId: 0021500171, Description: Anderson FLAGRANT.FOUL.TYPE1 (P2.PN) (M.Ayotte), Time: 2:02, EventNum: 242>

In [16]:
get_num_fta_from_foul(ne)

2

In [61]:
event.next_event.next_event.next_event


<StatsFreeThrow GameId: 0021700054, Description: MISS Jordan Free Throw 1 of 1, Time: 6:26, EventNum: 59>